## 1. merge

In [1]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [3]:
#csv 합치기
import glob
import os

input_file = r'./data3'
output_file = r'./data3/all_bus_per_time.csv'

allFile_list = glob.glob(os.path.join(input_file, 'bus_per_time_*'))
print(allFile_list)
allData = []
for file in allFile_list:
    df = pd.read_csv(file)
    allData.append(df)

dataCombine = pd.concat(allData, axis=0, ignore_index=True)
dataCombine.to_csv(output_file, index=False)

['./data3\\bus_per_time_202006.csv', './data3\\bus_per_time_202007.csv', './data3\\bus_per_time_202008.csv', './data3\\bus_per_time_202009.csv', './data3\\bus_per_time_202010.csv', './data3\\bus_per_time_202011.csv', './data3\\bus_per_time_202012.csv', './data3\\bus_per_time_202101.csv', './data3\\bus_per_time_202102.csv', './data3\\bus_per_time_202103.csv', './data3\\bus_per_time_202104.csv', './data3\\bus_per_time_202105.csv', './data3\\bus_per_time_202106.csv', './data3\\bus_per_time_202107.csv', './data3\\bus_per_time_202108.csv', './data3\\bus_per_time_202109.csv', './data3\\bus_per_time_202110.csv']


In [15]:
#합친 csv 불러오기
df = pd.read_csv("./data3/all_bus_per_time.csv")
df

,날짜,지역,항목,시간대,시간대평균(명),전월시간대평균(명),전월대비(명) ±
0,202006,서울특별시 강남구,승차인원,시간당 평균,13597,12025,1572
1,202006,서울특별시 강동구,승차인원,시간당 평균,4640,4197,443
2,202006,서울특별시 강북구,승차인원,시간당 평균,7777,7336,441
3,202006,서울특별시 강서구,승차인원,시간당 평균,7372,6785,587
4,202006,서울특별시 관악구,승차인원,시간당 평균,11719,10799,920
...,...,...,...,...,...,...,...
9345,202110,서울특별시 용산구,승차인원,00-01시,375,285,90
9346,202110,서울특별시 은평구,승차인원,00-01시,462,419,43
9347,202110,서울특별시 종로구,승차인원,00-01시,894,767,127
9348,202110,서울특별시 중구,승차인원,00-01시,628,532,97


## 2. 데이터 전처리

In [16]:
#데이터타입 확인
df.dtypes

날짜             int64
지역            object
항목            object
시간대           object
시간대평균(명)       int64
전월시간대평균(명)     int64
전월대비(명) ±      int64
dtype: object

In [17]:
#필요없는 컬럼 제거
df = df.drop(["항목","전월시간대평균(명)", "전월대비(명) ±"], axis=1)
df

,날짜,지역,시간대,시간대평균(명)
0,202006,서울특별시 강남구,시간당 평균,13597
1,202006,서울특별시 강동구,시간당 평균,4640
2,202006,서울특별시 강북구,시간당 평균,7777
3,202006,서울특별시 강서구,시간당 평균,7372
4,202006,서울특별시 관악구,시간당 평균,11719
...,...,...,...,...
9345,202110,서울특별시 용산구,00-01시,375
9346,202110,서울특별시 은평구,00-01시,462
9347,202110,서울특별시 종로구,00-01시,894
9348,202110,서울특별시 중구,00-01시,628


In [36]:
#필요없는 컬럼값 제거
df2 = df[df.시간대 != "시간당 평균"].reset_index(drop=True)
df2

,날짜,지역,시간대,시간대평균(명)
0,202006,서울특별시 강남구,04-05시,895
1,202006,서울특별시 강동구,04-05시,554
2,202006,서울특별시 강북구,04-05시,2195
3,202006,서울특별시 강서구,04-05시,1319
4,202006,서울특별시 관악구,04-05시,1712
...,...,...,...,...
8920,202110,서울특별시 용산구,00-01시,375
8921,202110,서울특별시 은평구,00-01시,462
8922,202110,서울특별시 종로구,00-01시,894
8923,202110,서울특별시 중구,00-01시,628


In [37]:
#컬럼명 이름 변경
df2 = df2.rename(columns={"시간대평균(명)":"평균승차인원"})
df2

,날짜,지역,시간대,평균승차인원
0,202006,서울특별시 강남구,04-05시,895
1,202006,서울특별시 강동구,04-05시,554
2,202006,서울특별시 강북구,04-05시,2195
3,202006,서울특별시 강서구,04-05시,1319
4,202006,서울특별시 관악구,04-05시,1712
...,...,...,...,...
8920,202110,서울특별시 용산구,00-01시,375
8921,202110,서울특별시 은평구,00-01시,462
8922,202110,서울특별시 종로구,00-01시,894
8923,202110,서울특별시 중구,00-01시,628


In [38]:
#날짜 데이트 타입변형
#이유 : int타입이므로 그룹화했을 때 날짜도 평균으로 나타남. 그러므로 string으로 바꿔줌.
df2 = df2.astype({"날짜" : str})
df2.dtypes

날짜        object
지역        object
시간대       object
평균승차인원     int64
dtype: object

In [39]:
#지역과 시간대별로 그룹화하기
df3 = df2.groupby(["지역","시간대"]).mean().round(0).reset_index()
df3

,지역,시간대,평균승차인원
0,서울특별시 강남구,00-01시,1499.0
1,서울특별시 강남구,04-05시,655.0
2,서울특별시 강남구,05-06시,2366.0
3,서울특별시 강남구,06-07시,5447.0
4,서울특별시 강남구,07-08시,13356.0
...,...,...,...
520,서울특별시 중랑구,19-20시,7228.0
521,서울특별시 중랑구,20-21시,4786.0
522,서울특별시 중랑구,21-22시,4397.0
523,서울특별시 중랑구,22-23시,3489.0


In [40]:
#정렬
df3 = df3.sort_values(["지역", "시간대"]).reset_index(drop=True)
df3

,지역,시간대,평균승차인원
0,서울특별시 강남구,00-01시,1499.0
1,서울특별시 강남구,04-05시,655.0
2,서울특별시 강남구,05-06시,2366.0
3,서울특별시 강남구,06-07시,5447.0
4,서울특별시 강남구,07-08시,13356.0
...,...,...,...
520,서울특별시 중랑구,19-20시,7228.0
521,서울특별시 중랑구,20-21시,4786.0
522,서울특별시 중랑구,21-22시,4397.0
523,서울특별시 중랑구,22-23시,3489.0


In [41]:
#parqeut 저장
df3.to_parquet("bus_per_time.parquet")